# 마케팅정보 EDA

In [1]:
import sys
sys.path.append('../utils')

In [2]:
from sklearn.pipeline import Pipeline
from sklearn import set_config
from preprocessing import (
    DateElapsedTransformer, NumericFeaturePreprocessor, ObjectFeaturePreprocessor, NumericTypeOptimizer,
)

pre_optimizer = NumericTypeOptimizer(mode='pre')
date_transformer = DateElapsedTransformer()
numeric_transformer = NumericFeaturePreprocessor()
object_transformer = ObjectFeaturePreprocessor()
post_optimizer = NumericTypeOptimizer(mode='post')

preprocessing_pipeline = Pipeline([
    ('pre_optimizer', pre_optimizer),
    ('object', object_transformer),
    ('date', date_transformer),
    ('numeric', numeric_transformer),
    ('post_optimizer', post_optimizer),
])

# Set display configuration to visualize the pipeline
set_config(display='diagram')

# Display the pipeline
preprocessing_pipeline

Pipeline(steps=[('pre_optimizer', NumericTypeOptimizer(mode='pre')),
                ('object',
                 ObjectFeaturePreprocessor(exclude_columns=['ID', 'Segment'])),
                ('date',
                 DateElapsedTransformer(base_date=Timestamp('2019-01-01 00:00:00'),
                                        exclude_columns=['ID', 'Segment',
                                                         '기준년월'])),
                ('numeric',
                 NumericFeaturePreprocessor(exclude_columns=['기준년월'])),
                ('post_optimizer', NumericTypeOptimizer())])

In [3]:
from sklearn.preprocessing import LabelEncoder
from data_loader import (
    load_data,
    load_segment
)

marketing_df, marketing_test_df = load_data("7.마케팅정보")

X = marketing_df.drop(columns=['ID', '기준년월'])
X = preprocessing_pipeline.fit_transform(X)
X_test = preprocessing_pipeline.transform(marketing_test_df.drop(columns=['ID', '기준년월']))

y = load_segment()
le = LabelEncoder()
y_encoded = le.fit_transform(y)

✅ File: ../../dataset/train/7.마케팅정보\201807_train_마케팅정보.parquet Completed!
✅ File: ../../dataset/train/7.마케팅정보\201808_train_마케팅정보.parquet Completed!
✅ File: ../../dataset/train/7.마케팅정보\201809_train_마케팅정보.parquet Completed!
✅ File: ../../dataset/train/7.마케팅정보\201810_train_마케팅정보.parquet Completed!
✅ File: ../../dataset/train/7.마케팅정보\201811_train_마케팅정보.parquet Completed!
✅ File: ../../dataset/train/7.마케팅정보\201812_train_마케팅정보.parquet Completed!
🔹 Shape : (2400000, 64)

✅ File: ../../dataset/test/7.마케팅정보\201807_test_마케팅정보.parquet Completed!
✅ File: ../../dataset/test/7.마케팅정보\201808_test_마케팅정보.parquet Completed!
✅ File: ../../dataset/test/7.마케팅정보\201809_test_마케팅정보.parquet Completed!
✅ File: ../../dataset/test/7.마케팅정보\201810_test_마케팅정보.parquet Completed!
✅ File: ../../dataset/test/7.마케팅정보\201811_test_마케팅정보.parquet Completed!
✅ File: ../../dataset/test/7.마케팅정보\201812_test_마케팅정보.parquet Completed!
🔹 Shape : (600000, 64)
Numeric Type Optimizer Transforming...
🧠 [mode=pre] 메모리 최적화: 1520.89 MB → 55

## Feature Selection

In [4]:
from feature import FeatureSelector

feature_engineering_pipeline = Pipeline([
    ('feature_selector', FeatureSelector()),
    ('optimize', post_optimizer),
])

X = feature_engineering_pipeline.fit_transform(X, y_encoded)
X_test = feature_engineering_pipeline.transform(X_test)

🎯 Fitting RandomForest for Feature Selection...
✅ Selected 9 features out of 47
📊 Top Selected Features by Importance:
1) 	컨택건수_이용유도_EM_B0M (0.1684)
2) 	컨택건수_이용유도_청구서_B0M (0.1030)
3) 	컨택건수_카드론_TM_R6M (0.0220)
4) 	컨택건수_이용유도_TM_R6M (0.1511)
5) 	컨택건수_보험_TM_R6M (0.0283)
6) 	컨택건수_이용유도_LMS_R6M (0.0549)
7) 	컨택건수_이용유도_EM_R6M (0.1832)
8) 	컨택건수_이용유도_청구서_R6M (0.0783)
9) 	컨택건수_이용유도_인터넷_R6M (0.0607)
Numeric Type Optimizer Transforming...
🧠 [mode=post] 메모리 최적화: 20.60 MB → 20.60 MB (0.0% 감소)
Numeric Type Optimizer Transforming...
🧠 [mode=post] 메모리 최적화: 5.15 MB → 5.15 MB (0.0% 감소)


c:\Users\Lee JinSeoung\OneDrive\Desktop\4학년 1학기\고급프로그래밍\과제\Credit-Card-Segment-Classfication\.venv\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
c:\Users\Lee JinSeoung\OneDrive\Desktop\4학년 1학기\고급프로그래밍\과제\Credit-Card-Segment-Classfication\.venv\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [5]:
import pandas as pd

train_df = pd.concat([marketing_df[['ID', '기준년월']], X], axis=1)
test_df = pd.concat([marketing_test_df[['ID', '기준년월']], X_test], axis=1)

train_df.to_parquet('../../dataset/train/marketing.parquet', index=False)
test_df.to_parquet('../../dataset/test/marketing.parquet', index=False)